In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.assistant.rag_helpers import similarity_search_for_tenant
from src.assistant.vector_db import get_embedding_model

In [2]:
# Test query
test_query = "how to release residues from regulatory control"

# Get results
results = similarity_search_for_tenant(
    tenant_id="default",
    embed_llm=get_embedding_model(),
    persist_directory="../database",
    similarity="cosine",
    normal=True,
    query=test_query
)

# Print results
results

/home/he/ai/dev/langgraph/local-rag-researcher-deepseek-he/src/assistant/vector_db.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_kwargs={'device': 'cpu'})
/home/he/ai/dev/langgraph/local-rag-researcher-deepseek-he/src/assistant/vector_db.py:11: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  return HuggingFaceEmbeddings(model_kwargs={'device': 'cpu'})
/home/he/ai/dev/langgraph/local-rag-researcher-deepseek-he/lrrd-venv/lib/python3.12/s

CUDA memory cache cleared


/home/he/ai/dev/langgraph/local-rag-researcher-deepseek-he/src/assistant/rag_helpers.py:91: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


CUDA memory cache cleared


[Document(metadata={'id': 'strlschg_en_bf.pdf:0:102', 'page': 0, 'source': 'strlschg_en_bf.pdf'}, page_content='methods of recovery and disposal laid down by a statutory ordinance in accordance with the second sentence are complied with in their disposal or recovery. the federal government is herewith empowered to issue a statutory ordinance, with the consent of the bundesrat, es\xad tablishing the monitoring limits and methods of recovery and disposal to apply to residues. 3 residues accumulated may not be mixed or diluted prior to their intended disposal or recovery in order to satisfy the monitoring limits in accordance with subsection 2. the first sentence shall also apply to residues accumulated abroad and brought into germany for recovery. 4 if the residues requiring monitoring are stored on the operating premises of the par\xad ty subject to an obligation in accordance with subsection 1, the latter must declare said storage to the competent authority. the competent authority sha

In [3]:
def transform_documents(documents):
    """
    Transforms a list of Document objects into a specific dictionary format.
    
    Args:
        documents (list): List of Document objects with metadata and page_content
        
    Returns:
        list: List of dictionaries with content and metadata in the required format
    """
    samples = []
    
    for doc in documents:
        transformed_doc = {
            "content": doc.page_content,
            "metadata": {
                "name": doc.metadata['id'],
                "path": doc.metadata['source']
            }
        }
        samples.append(transformed_doc)
    
    return samples

In [4]:
tr=transform_documents(results)
tr

[{'content': 'methods of recovery and disposal laid down by a statutory ordinance in accordance with the second sentence are complied with in their disposal or recovery. the federal government is herewith empowered to issue a statutory ordinance, with the consent of the bundesrat, es\xad tablishing the monitoring limits and methods of recovery and disposal to apply to residues. 3 residues accumulated may not be mixed or diluted prior to their intended disposal or recovery in order to satisfy the monitoring limits in accordance with subsection 2. the first sentence shall also apply to residues accumulated abroad and brought into germany for recovery. 4 if the residues requiring monitoring are stored on the operating premises of the par\xad ty subject to an obligation in accordance with subsection 1, the latter must declare said storage to the competent authority. the competent authority shall be informed without undue delay if such storage ceases. 5 for those residues that are not subje

In [5]:
import ollama
import re
from IPython.display import Markdown, display

In [6]:
def source_summarizer(context_documents, llm_model="deepseek-r1"):
    system_message = """
    You are an expert summarizer working within a RAG system. Your task is to create a concise, accurate summary of the provided information while properly attributing all facts to their sources.

    Guidelines:
    - Create a clear, coherent summary limited to 3-5 sentences
    - Focus on the most important facts and insights
    - Maintain factual accuracy without adding new information
    - Use neutral, professional language
    - Cite EVERY piece of information using the format [Document Name](document_path)
    - Place citations immediately after the relevant information
    - Ensure each citation is correctly matched to its source
    - Return only the plain text summary without markdown formatting
    """

    formatted_context = "\n".join(
        f"Content: {doc['content']}\nSource: {doc['metadata']['name']}\nPath: {doc['metadata']['path']}"
        for doc in context_documents
    )

    prompt = f"""
    Here are the documents to summarize:
    
    {formatted_context}
    
    Provide a concise summary with proper citations:
    """
    
    response = ollama.chat(
        model=llm_model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt}
        ]
    )
    
    response_content = response["message"]["content"]
    
    # Clean markdown formatting if present
    try:
        final_content = re.sub(r"<think>.*?</think>", "", response_content, flags=re.DOTALL).strip()
    except:
        final_content = response_content.strip()

    # Extract metadata from all documents
    document_names = [doc['metadata']['name'] for doc in context_documents]
    document_paths = [doc['metadata']['path'] for doc in context_documents]

    return {
        "content": final_content,
        "metadata": {
            "name": document_names,
            "path": document_paths
        }
    }

In [7]:
smry = source_summarizer(tr)
smry

{'content': "The federal government has established an ordinance requiring the declaration of residues' nature, mass, and specific activity when monitoring is necessary [102: strlschg_en_bf.pdf:0:102]. Residues cannot be mixed or diluted before disposal if they require monitoring to ensure public protection. Cross-border transportation of monitored residues into Germany for disposal is prohibited unless specific safety measures are in place, ensuring the public and occupational workers' safety [103: strlschg_en_bf.pdf:0:103]. Parties under an obligation must declare storage requirements without undue delay, while non-monitored residues may require additional technical procedures as outlined by the competent authority.",
 'metadata': {'name': ['strlschg_en_bf.pdf:0:102', 'strlschg_en_bf.pdf:0:103'],
  'path': ['strlschg_en_bf.pdf', 'strlschg_en_bf.pdf']}}

In [8]:
display(Markdown(smry['content']))

The federal government has established an ordinance requiring the declaration of residues' nature, mass, and specific activity when monitoring is necessary [102: strlschg_en_bf.pdf:0:102]. Residues cannot be mixed or diluted before disposal if they require monitoring to ensure public protection. Cross-border transportation of monitored residues into Germany for disposal is prohibited unless specific safety measures are in place, ensuring the public and occupational workers' safety [103: strlschg_en_bf.pdf:0:103]. Parties under an obligation must declare storage requirements without undue delay, while non-monitored residues may require additional technical procedures as outlined by the competent authority.

In [9]:
models = ["deepseek-r1:latest", "deepseek-r1:70b", "qwq", "gemma3:27b", "mistral-small:latest", "llama3.3", "llama3.2"]

for model in models:
    summary = source_summarizer(tr,model)
    display(Markdown(f'#LLM: {model}'))
    display(Markdown(summary['content']))
    print('________________________________')

#LLM: deepseek-r1:latest

The German federal government has authorized a statutory ordinance empowering the Bundestag to establish monitoring limits for residues, ensuring they are not mixed or diluted prior to disposal or recovery [1](https://strlschg_en_bf.pdf:0:102). Residues accumulated on a party's premises must be declared to the competent authority immediately if monitoring is required. For residues exempt from monitoring, additional technical procedures may be established by the authority. Cross-border transportation of residues into Germany for disposal is prohibited [1](https://strlschg_en_bf.pdf:0:102). Residues requiring monitoring must declare their intended recovery or disposal to the competent authority promptly if their nature, mass, and specific activity are known. Release from monitoring obligations is permissible only if measures ensure public protection and comply with occupational exposure limits [2](https://strlschg_en_bf.pdf:0:103). Public protection criteria require an effective dose of 1 msv for disposal or recovery.

________________________________


#LLM: deepseek-r1:70b

The federal government is empowered to issue statutory ordinances, with the consent of the Bundesrat, establishing methods and limits for the recovery or disposal of residues [strlschg_en_bf.pdf:0:102](strlschg_en_bf.pdf). Residues requiring monitoring must not be mixed or diluted to meet these limits and must be stored securely to prevent unauthorized access [strlschg_en_bf.pdf:0:102](strlschg_en_bf.pdf). Cross-border transportation of residues into Germany for disposal is prohibited unless approved by the competent authority, which may require proof of compliance with specified recovery or disposal methods [strlschg_en_bf.pdf:0:102](strlschg_en_bf.pdf). The party under obligation must declare storage details to the authority and ensure residues are only passed on for disposal or recovery [strlschg_en_bf.pdf:0:103](strlschg_en_bf.pdf). Release from monitoring may be granted if public exposure is adequately protected, occupational doses remain within limits, and waste laws are complied with [strlschg_en_bf.pdf:0:103](strlschg_en_bf.pdf).

...

The federal government is empowered to issue a statutory ordinance establishing monitoring limits and methods of recovery and disposal for residues, with the consent of the Bundesrat [strlschg_en_bf.pdf](strlschg_en_bf.pdf:0:102). Residues accumulated may not be mixed or diluted prior to disposal or recovery to satisfy monitoring limits [strlschg_en_bf.pdf](strlschg_en_bf.pdf:0:102). Parties subject to an obligation must declare storage of residues to the competent authority and secure them against loss and unauthorized access [strlschg_en_bf.pdf](strlschg_en_bf.pdf:0:102). The cross-border transportation of residues into Germany for disposal is prohibited [strlschg_en_bf.pdf](strlschg_en_bf.pdf:0:102). The competent authority may release residues from monitoring for a particular recovery or disposal if certain criteria are met, including ensuring requisite protection of the population and complying with waste law [strlschg_en_bf.pdf](strlschg_en_bf.pdf:0:103).

________________________________


#LLM: llama3.2

The German Federal Ordinance on the Release of Residues from Monitoring Empowerment establishes procedures for the release of residues from monitoring requirements. A party subject to an obligation must declare their intention to recover or dispose of residues to the competent authority without undue delay, stating their nature, mass, and specific activity [1]. The competent authority may release residues requiring monitoring if certain conditions are met, including ensuring public protection against exposure, limiting occupational worker exposure, and compliance with relevant waste laws [2]. This allows for controlled recovery or disposal under specific circumstances.

[1] strlschg_en_bf.pdf:0:102
[2] strlschg_en_bf.pdf:0:103

________________________________
